In [ ]:
import os
import librosa
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
def extract_mfcc_with_deltas(audio, sr=16000, n_mfcc=13):
    """
    Extract MFCCs, delta, and delta-delta features from an audio signal.
    """
    hop_length = int(0.01 * sr)  # 10 ms hop length
    win_length = int(0.025 * sr)  # 25 ms window length
    
    # Calculate MFCCs (including the 0th coefficient)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, win_length=win_length, window='hamming')
    
    # Calculate log energy feature
    # energy = np.log(np.sum(librosa.feature.rms(y=audio, hop_length=hop_length)**2, axis=0))
    
    # Append log energy as the 0th feature
    # mfccs[0, :] = energy
    
    # Compute the deltas (first derivative)
    mfcc_delta = librosa.feature.delta(mfccs)
    
    # Compute delta-delta (second derivative)
    mfcc_delta2 = librosa.feature.delta(mfccs, order=2)
    
    # Stack MFCCs, delta, and delta-delta features
    mfcc_combined = np.vstack([mfccs, mfcc_delta, mfcc_delta2])
    
    return mfcc_combined

In [3]:
# Function to load data from folders
def load_data(data_dir):
    X = []
    y = []
    labels = os.listdir(data_dir)
    
    for label in labels:
        label_dir = os.path.join(data_dir, label)
        for file_name in os.listdir(label_dir):
            if file_name.endswith(".wav"):
                file_path = os.path.join(label_dir, file_name)
                # Load the audio file
                audio, sr = librosa.load(file_path, sr=None)
                features = extract_mfcc_with_deltas(audio,sr)
                X.append(features)
                y.append(label)
    
    return np.array(X), np.array(y)


# Load training and testing data
train_data_dir = r'C:\Users\USER\Downloads\SV_NCKH_audio_event\Train'
test_data_dir = r'C:\Users\USER\Downloads\SV_NCKH_audio_event\Test'

X_train, y_train = load_data(train_data_dir)
X_test, y_test = load_data(test_data_dir)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Reshape for CNN input (add channel dimension)
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]



In [4]:
# Build CNN model
def create_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential()

    # 2D Convolution Layer
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Additional Conv Layers
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Flatten and Fully Connected Layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))

    # Output Layer
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    # Create the Adam optimizer with the custom learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
# Get input shape and number of classes
input_shape = X_train.shape[1:]  # shape of each input sample
num_classes = len(np.unique(y_train))  # number of class labels

# Create and train the CNN model
model = create_cnn_model(input_shape, num_classes)
model.summary()

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 37, 99, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 18, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 23, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 11776)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,507,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 21)             │         2,709 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,528,981 (5.83 MB)

 Trainable params: 1,528,981 (5.83 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.1732 - loss: 6.3476 - val_accuracy: 0.4636 - val_loss: 1.7317
Epoch 2/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.5811 - loss: 1.3573 - val_accuracy: 0.5254 - val_loss: 1.5903
Epoch 3/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.7071 - loss: 0.8831 - val_accuracy: 0.6173 - val_loss: 1.4292
Epoch 4/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.8009 - loss: 0.6289 - val_accuracy: 0.6118 - val_loss: 1.6379
Epoch 5/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.8417 - loss: 0.4743 - val_accuracy: 0.6118 - val_loss: 1.9390
Epoch 6/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.8941 - loss: 0.3354 - val_accuracy: 0.5898 - val_loss: 2.0057
Epoch 7/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - accuracy: 0.9144 - loss: 0.2458 - val_accuracy: 0.5789 - val_loss: 2.1293
Epoch 8/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.9547 - loss: 0.1479 - val_accuracy: 0.